# DD2424 - Project 3 - NLP

> Group 12: Tristan Perrot Paul Mauduit Adrien Jouanny Arthur Depret


## Imports


In [1]:
!pip install kaggle
!pip install nlpaug
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 5.8 MB/s eta 0:00:00


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes


In [2]:
!kaggle competitions download -c nlp-getting-started
!kaggle datasets download -d rtatman/glove-global-vectors-for-word-representation
! unzip /content/glove-global-vectors-for-word-representation.zip
import math
import re
import os
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from nltk.corpus import gutenberg
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.notebook import trange, tqdm
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

nltk.download('gutenberg')

403 - Forbidden - You must accept this competition's rules before you'll be able to download files.
Dataset URL: https://www.kaggle.com/datasets/rtatman/glove-global-vectors-for-word-representation
License(s): other
 97% 443M/458M [00:04<00:00, 98.3MB/s]
100% 458M/458M [00:04<00:00, 99.9MB/s]
Archive:  /content/glove-global-vectors-for-word-representation.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.50d.txt        


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [3]:
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_properties(i).name)

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

## Data

We used 3 relevant books from the Gutenberg project:

- ["Alice's Adventures in Wonderland" by Lewis Carroll](https://www.gutenberg.org/ebooks/11)
- ["Twenty Thousand Leagues under the Sea" by Jules Verne](https://www.gutenberg.org/ebooks/164)
- ["The Joyful Wisdom" by Friedrich Nietzsche](https://www.gutenberg.org/ebooks/52881)


In [4]:
# Load the data from the gutenberg corpus
alice = gutenberg.raw('carroll-alice.txt')

# Remove the header
header = alice.find("CHAPTER I.")
alice = alice[header:]

# Set to lowercase
alice = alice.lower()
alice = alice.replace('\'','')
# Split the dataset into training, validation and test sets
train = alice[:-22067]
val = alice[-22067:-11681]
test = alice[-11681:]

print(alice[:100])

chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on t


### Preprocessing


In [5]:
# Remove \n characters
train = train.replace("\n", " ")

# Split the text based on ., ,, :, ;
sentances = re.split(r'[.,:;]\s*', train)

In [6]:
aug_w2v = naw.WordEmbsAug(model_type='glove', model_path='/content/glove.6B.100d.txt', action="substitute")
aug_w2v.aug_p = 0.2
n_rewrite = 2

In [ ]:
# print(sentances)
# print(len(sentances))

In [ ]:
# Augment sentences and collect them in the placeholder
augmented_sentances = []
for i in tqdm(range(len(sentances))):
    for ii in range(n_rewrite):
        augmented_sentances += aug_w2v.augment(sentances[i])


  0%|          | 0/3204 [00:00<?, ?it/s]

In [ ]:
print(len(sentances))
print(len(augmented_sentances))
augmented_train = ''
for i in range(len(sentances)):
  augmented_train += ' ' + augmented_sentances[2*i]
for i in range(len(sentances)):
  augmented_train += ' ' + augmented_sentances[2*i+1]
#print(augmented_train)
alice_augmented = alice[:-22067] + augmented_train + val + test
with open('augmented_alice.txt', 'w') as f:
    f.write(alice_augmented)